In [34]:
from imp import reload
import functions as fun
# Change in fun
reload(fun)

<module 'functions' from '/home/carlos/Documents/MachineLearning/manning/calvar-nlp-entity-linking-lp-main/milestone5/functions.py'>

In [2]:
import pandas as pd

In [3]:
#Clean data
data_df = fun.remove_missing_add_length("../data/mtsamples_descriptions_clean.csv", "data_df.csv");

In [4]:
data_df.head()

,id,id_description,medical_specialty_new,text,year,borough,length
0,0,0,Gastroenterology,EGD with photos and biopsies,2013.0,Merton,29
1,1,0,Gastroenterology,This is a 75-year-old female who presents wit...,2013.0,Merton,106
2,2,0,Gastroenterology,She has a previous history of hiatal hernia,2013.0,Merton,44
3,3,0,Gastroenterology,She was on Prevacid currently,2013.0,Merton,30
4,4,1,Urology,"Pelvic tumor, cystocele, rectocele, and uteri...",2013.0,Harrow,56


In [7]:
#Create list with ordered dictionaries of rows
sentences = fun.create_ordered_dicts(data_df)

In [8]:
sentences[0]

OrderedDict([('id', '0'),
             ('id_description', '0'),
             ('medical_specialty_new', ' Gastroenterology'),
             ('text', ' EGD with photos and biopsies'),
             ('year', 2013.0),
             ('borough', 'Merton'),
             ('length', 29)])

In [9]:
#Initialize the debater clients to use
arg_quality_client, keypoints_client, term_wikifier_client = fun.init_quality_keypoints("../APIkey.txt")

In [43]:
#Set a topic for the argument quality service
topic = "Left heart catheterization, left ventriculography, coronary angiography, and successful stenting of tight lesion in the distal circumflex and moderately tight lesion in the mid-right coronary artery."
Out = open("topic.txt", "w")
Out.write(topic)
Out.close()

In [11]:
#Use the argument quality client to assign scores to the relation of the texts in the database and the topic.
# Return the top 1000 scored items.
sentences_top_1000_aq = fun.top_1000_sentences(arg_quality_client, topic, sentences)

ArgumentQualityClient: 100%|████████████████| 3245/3245 [00:36<00:00, 88.02it/s]


In [12]:
sentences_top_1000_aq[0]

OrderedDict([('id', '843'),
             ('id_description', '863'),
             ('medical_specialty_new', ' Cardiovascular / Pulmonary'),
             ('text',
              ' Cardiac Catheterization - An obese female with a family history of coronary disease and history of chest radiation for Hodgkin disease, presents with an acute myocardial infarction with elevated enzymes.'),
             ('year', 2010.0),
             ('borough', 'Lewisham'),
             ('length', 204)])

In [13]:
#Create structures storing the run parameters, sentences text and sentences ids
run_params, sentences_texts, sentences_ids = fun.create_sentence_structs(0.98, 5, sentences_top_1000_aq)

In [14]:
#Run the key points analysis
kpa_result, job_id = fun.key_point_analysis(keypoints_client, "medical_demo2", run_params, sentences_texts, sentences_ids)

In [15]:
job_id

'62dd689b8e7da8a7796aa4d9'

In [16]:
print(kpa_result["keypoint_matchings"][0]["matching"][0])

{'domain': 'medical_demo', 'comment_id': '1220', 'sentence_id': 0, 'sents_in_comment': 1, 'span_start': 0, 'span_end': 157, 'num_tokens': 26, 'argument_quality': 0.650917649269104, 'sentence_text': '   The patient is a 1-year-old male with a history of chronic otitis media with effusion and conductive hearing loss refractory to outpatient medical therapy', 'score': 0}


In [23]:
#Convert kpa result to a dataframe
match_df =  fun.kpa_to_dataframe(kpa_result)

In [24]:
match_df.tail()

,kp,sentence_text,match_score,comment_id,sentence_id,sents_in_comment,span_start,span_end,num_tokens,argument_quality
1,Possible inflammatory bowel disease,Laparoscopic right partial nephrectomy due to...,0.981618,2014,0,1,0,64,9,0.523238
2,Risk factors for coronary heart disease.,Mild-to-moderate obstructive ventilatory impa...,0.980484,931,0,1,0,52,6,0.570248
3,"Right shoulder impingement syndrome, right su...",Cervical spondylosis and kyphotic deformity,0.980421,2899,0,1,0,44,5,0.534918
4,Urgent cardiac catheterization with coronary ...,The internal carotid artery had good backflow...,0.980095,2242,0,1,0,62,9,0.463009
5,Hemoptysis and history of lung cancer,"Chronic pelvic pain, probably secondary to e...",0.980687,1688,0,1,0,59,7,0.584735


In [25]:
#Merge the kpa results with the phrases from the dataset and store it in file
fun.merge_kpa(match_df, data_df, "merge_df.csv")

In [26]:
merge_df = pd.read_csv("merge_df.csv")
merge_df.tail()

,kp,sentence_text,match_score,comment_id,sentence_id,sents_in_comment,span_start,span_end,num_tokens,argument_quality,id,id_description,medical_specialty_new
1,Possible inflammatory bowel disease,Laparoscopic right partial nephrectomy due to...,0.981618,2014,0,1,0,64,9,0.523238,2014,1516,Nephrology
2,Risk factors for coronary heart disease.,Mild-to-moderate obstructive ventilatory impa...,0.980484,931,0,1,0,52,6,0.570248,931,924,Cardiovascular / Pulmonary
3,"Right shoulder impingement syndrome, right su...",Cervical spondylosis and kyphotic deformity,0.980421,2899,0,1,0,44,5,0.534918,2899,2079,Orthopedic
4,Urgent cardiac catheterization with coronary ...,The internal carotid artery had good backflow...,0.980095,2242,0,1,0,62,9,0.463009,2242,1626,Cardiovascular / Pulmonary
5,Hemoptysis and history of lung cancer,"Chronic pelvic pain, probably secondary to e...",0.980687,1688,0,1,0,59,7,0.584735,1688,1352,Obstetrics / Gynecology


In [32]:
#Tally the references in wikipedia for each key point
terms = fun.get_wikipedia_terms(term_wikifier_client, merge_df)

TermWikifierClient: 100%|█████████████████████████| 1/1 [00:00<00:00,  1.03it/s]


In [36]:
#dump to json
fun.dump_to_json(terms, "terms.json")

In [39]:
for k in terms:
    print(k)

none
 Right shoulder impingement syndrome, right suprascapular neuropathy.
 Urgent cardiac catheterization with coronary angiogram.
 Possible inflammatory bowel disease
 Hemoptysis and history of lung cancer
 Risk factors for coronary heart disease.


In [40]:
#Display the analysis for one diagnosis
df_check = pd.DataFrame(
    list(terms[" Possible inflammatory bowel disease"].items()),
    columns = ["Term", "Count"],
).sort_values(by="Count", ascending=False)

df_check.head(10)

,Term,Count
0,Nephrectomy,1
1,Laparoscopy,1
2,Kidney tumour,1


In [41]:
#Remove rows with keypoint 'none'
clean_df = merge_df[merge_df["kp"] != "none"]

In [42]:
#Save to file
clean_df.to_csv("clean_df.csv", index=False)

### Run the dashboard with `streamlit run milestone5.py`